In [ ]:
from ultralytics import YOLO
from queue import Queue
import cv2
import time
import threading

In [ ]:
que = Queue()

def for_thread(model, size, start, video, maxSize):
    times = time.time()
    results = []
    end = 0
    for i in range(start, start + size, 1):
        if i == maxSize:
            break
        result = model(video[i])
        res = result[0]
        keypoints = res.keypoints
        results.append(res.plot())
        end = i
    ts = time.time()
    que.put(results)
    print(ts - times)
    return end

In [ ]:
# times = time.time()
cap = cv2.VideoCapture('chloe.mp4')
model = YOLO('yolov8n-pose.pt')
video = []

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 29.97, (1280, 720))

while True:
    r, frame = cap.read()
    if not r:
        break
    video.append(frame)
cap.release()

In [ ]:
numOfThreads = 17
models = []
resultss = []
sizes = []
threads = []

lens = len(video)

for i in range(0, numOfThreads):
    model = YOLO('yolov8n-pose.pt')
    models.append(model)
    sizes.append(lens//numOfThreads)
    if lens%numOfThreads != 0 and i < lens%numOfThreads:
        sizes[i] += 1

start = 0

In [ ]:
times = time.time()
for i in range(len(models)):
    th = threading.Thread(target=for_thread, args=(models[i], sizes[i], start, video, lens,), daemon=True)
    threads.append(th)

for th in threads:
    th.start()

out.release()

In [ ]:

# for i in range(len(models)):
#     res, start = for_thread(models[i], sizes[i], start, video, lens)
#     resultss.append(res)

# for i in resultss:
#     for j in i:
#         out.write(j)
# out.release()